<a href="https://colab.research.google.com/github/nakamura196/000_tools/blob/main/NDLOCR_v2%E3%81%AE%E5%AE%9F%E8%A1%8C%E4%BE%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!--
<h3 style="color: red;">⚠️ 現在、不具合が発生しています</h3>
<p><b>本ノートブックは現在ご利用いただけません。<br/>復旧までしばらくお待ちください。</b></p>
-->

<!--
Google Colaboratoryのアップデートなどにより、本ノートブックが実行できない場合があります。その場合、以下の稼働状況をご確認ください。 https://ldas.statuspage.io/

<br/>

また成果の共有にご協力いただけますと幸いです。 <a href="https://mahalo.ex.nii.ac.jp/button/d02a35f0-c34c-4670-9c15-f6927797940f/mahalo">Mahalo Button</a> by DIAS (Data Integration and Analysis).

-->

<br/><br/>

# <font color="orange">NDLOCR（ver.2.1）の実行例</font>

本ノートブックのライセンス： <img src="http://i.creativecommons.org/p/zero/1.0/88x31.png" style="border-style: none;" alt="CC0" />

<br/>

NDL OCRの説明: https://github.com/ndl-lab/ndlocr_cli

<br/>

本ノートブックでは、Google Driveに認識結果を出力します。本ノートブックの説明については、以下を参考にしてください。

https://zenn.dev/nakamura196/articles/43151b473e8954

<br/>


## 参考にしたノートブック

@blue0620 さんが作成したノートブック

https://github.com/blue0620/NDLOCR-GoogleColabVersion/blob/main/NDLOCRv2_googlecolabversion.ipynb

<br/>

## 更新内容
- 2025-09-04
 - 不具合を修正しました。
- 2025-07-08
 - 不具合を修正しました。
- 2025-04-02
 - 不具合を修正しました。
- 2024-10-28
 - PDFを処理する際の不具合を修正しました。
- 2024-08-20
 - 不具合を修正しました。
- 2024-03-20
 - PDFを処理する際の画像のリサイズ処理を追加しました。
- 2024-03-19
 - 不具合を修正しました。
- 2023-10-23
 - ステータスページへのリンクを追加しました。
 - エラーのハンドリングを修正しました。
 - 不具合を修正しました。
- 2023-09-24
 - PDF出力時の不具合を修正しました。
- 2023-09-23
 - 横書きテキストに対するPDF出力の不具合を修正しました。
- 2023-09-19
 - ノートブックを公開しました。

<br/><br/><br/><br/>

## 1.初期セットアップ

少し時間がかかります。初回のみ実行が必要です。

またセットアップ中に「セッションを再起動する」という画面が表示された場合には、「キャンセル」を押してください。

In [ ]:
#@title セットアップ1
from pathlib import Path
CONTENT_DIR = str(Path("/content"))

from IPython.display import clear_output

# Google Drive関連
from google.colab import drive
drive.mount(f'{CONTENT_DIR}/drive/')

###

!pip install pip==24.0
!pip install numpy==1.26.4
!pip install --trusted-host download.pytorch.org torch==2.1.1 torchvision==0.16.1 torchaudio==2.1.1 torchtext==0.16.1 --index-url https://download.pytorch.org/whl/cu121

# Googleドライブのパスを取得
!pip install kora



%cd {CONTENT_DIR}

# !nvidia-smi
# !pip list | grep torch
!git clone --recursive https://github.com/ndl-lab/ndlocr_cli -b feature/colab
# #2. 必要なパッケージをインストールする
PROJECT_DIR=str(Path(f"{CONTENT_DIR}/ndlocr_cli"))
# !pip install -r {PROJECT_DIR}/requirements.txt # コメントアウト

!pip install hydra-colorlog
!pip install hydra-core
!pip install mmpretrain==1.2.0
!pip install pytorch-lightning==1.6.5
!pip install datasets

!pip install mmcv==2.1.0 -f https://download.openmmlab.com/mmcv/dist/cu121/torch2.1/index.html

!pip install torchmetrics==0.11.4

clear_output()

'''
%cd {PROJECT_DIR}/
!wget http://www.phontron.com/kytea/download/kytea-0.4.7.tar.gz
!tar xzvf kytea-0.4.7.tar.gz
%cd kytea-0.4.7
!./configure
!make
!make install
!ldconfig
%cd /content

!pip install kytea==0.1.7
'''

%cd {PROJECT_DIR}/submodules/ndl_layout
!git clone https://github.com/open-mmlab/mmdetection.git -b v3.3.0
%cd {PROJECT_DIR}/submodules/ndl_layout/mmdetection
#下行はGPUのメモリ不足になった場合にコメントアウトを外すとよい。
!sed -i -e 's/GPU_MEM_LIMIT = 1024\*\*3/GPU_MEM_LIMIT = 1024\*\*3\/\/5/' mmdet/models/roi_heads/mask_heads/fcn_mask_head.py
!python setup.py bdist_wheel
!pip install dist/*.whl

# %cd /content

# 出力結果のクリア
clear_output()

# # 4. OCRに必要な学習済みモデルをダウンロードする
%cd {PROJECT_DIR}
!wget -nc https://lab.ndl.go.jp/dataset/ndlocr_v2/text_recognition_lightning/resnet-orient2.ckpt -P ./submodules/text_recognition_lightning/models
!wget -nc https://lab.ndl.go.jp/dataset/ndlocr_v2/text_recognition_lightning/rf_author/model.pkl -P ./submodules/text_recognition_lightning/models/rf_author/
!wget -nc https://lab.ndl.go.jp/dataset/ndlocr_v2/text_recognition_lightning/rf_title/model.pkl -P ./submodules/text_recognition_lightning/models/rf_title/
!wget -nc https://lab.ndl.go.jp/dataset/ndlocr_v2/ndl_layout/ndl_retrainmodel.pth -P ./submodules/ndl_layout/models
!wget -nc https://lab.ndl.go.jp/dataset/ndlocr_v2/separate_pages_mmdet/epoch_180.pth -P ./submodules/separate_pages_mmdet/models

# %cd /content/

clear_output()

DOCS_DIR = f"{CONTENT_DIR}/docs"

from google.colab import output

PORT = 8001
!mkdir -p {DOCS_DIR}
%cd {DOCS_DIR}
!nohup python3 -m http.server $PORT > server.log 2>&1 &

if not Path("index.html").exists():
  !wget https://nakamura196.github.io/mirador-integration-textoverlay/index.html -O index.html
  !mkdir -p dist
  !wget https://nakamura196.github.io/mirador-integration-textoverlay/dist/main.js -O dist/main.js

# !apt-get install poppler-utils
!apt-get update && apt-get install -y poppler-utils
!pip install pdf2image
!pip install ldas==0.0.8
!pip install ocr_iiif_tools==0.0.17
!pip install scikit-learn==1.2.2

!pip uninstall -y torchao
!pip install transformers==4.45.2

clear_output()

In [ ]:
#@title セットアップ2

!pip install PyMuPDF

import fitz  # pip install PyMuPDF
from PIL import Image
import io
from pathlib import Path
import os

import uuid
import yaml
import datetime
import requests
import os
from pathlib import Path
from pdf2image import convert_from_path
import os
from google.colab import files
import glob
import xml.etree.ElementTree as ET
from ldas.iiif import IIIF
from ocr_iiif_tools.pdf import PdfClient
import subprocess
import json
from kora.xattr import get_id
from tqdm import tqdm
from PIL import Image

def message(path):
  print("\n\n以下に出力しました。\nhttps://drive.google.com/drive/folders/{}".format(get_id(path)))

# google colab依存
def upload_get_input_file():
  uploaded = files.upload()
  input_file = next(iter(uploaded))
  return input_file


def disable_ruby_reading_in_config():
  # 設定ファイル内でルビ読みを無効にします。
  config_path = f'{PROJECT_DIR}/config.yml'

  with open(config_path) as file:
      config = yaml.safe_load(file)
      config["ruby_read"] = False

      with open(config_path, 'w') as file:
        yaml.dump(config, file)

disable_ruby_reading_in_config()

class Task:
  docs_dir = "/content/docs"

  project_dir = "/content/ndlocr_cli"

  def __init__(self, output_dir, process, ruby, docs_dir = None, project_dir = None):
    # プロセス名を対応する範囲に変換します。
    self.convert_process_to_range(process)

    if docs_dir != None:
      self.docs_dir = docs_dir

    if project_dir != None:
      self.project_dir = project_dir

    self.files_dir = f"{self.docs_dir}/files"

    # プロセスIDを生成します。
    self.generate_process_id()

    # google driveに出力フォルダを作成する
    self.prepare_google_dirve_directory(f"{output_dir}/{self.task_id}")

    # docs/filesにtask_idに基づくフォルダを作成します。
    self.generate_docs_files_task_id_directory()

    # docs/files/{task_id}/img フォルダを作成します。
    self.prepare_img_directory()

    # OCRの出力フォルダを用意する
    output_tmp_dir = f"/content/output/{self.task_id}"
    self.prepare_output_directory(output_tmp_dir)

  def prepare_google_dirve_directory(self, path):
     # google driveに出力フォルダを作成する
    os.makedirs(path, exist_ok=True)
    self.google_drive_path = path

  def convert_process_to_range(self, process):
    # プロセス名を対応する範囲に変換します。
    process_ranges = {
        "傾き補正,レイアウト抽出,文字認識(OCR)": "1..3",
        "レイアウト抽出,文字認識(OCR)": "2..3",
        "文字認識(OCR)": "3",
    }
    self.process_range = process_ranges.get(process, "0..3")

  def generate_process_id(self):
    # プロセスIDを生成します。
    self.task_id = uuid.uuid1()

  def generate_docs_files_task_id_directory(self):
    # docs/filesにtask_idに基づくフォルダを作成します。
    self.temporary_directory = f"{self.files_dir}/{self.task_id}"

  def prepare_img_directory(self):
    # docs/files/{task_id}/img フォルダを作成します。
    target_dir_path = Path(self.temporary_directory)
    img_dir_path = target_dir_path / "img"
    img_dir_path.mkdir(parents=True, exist_ok=True)
    self.img_dir_path = str(img_dir_path)

  def prepare_output_directory(self, output_dir):
    output_dir_path = Path(output_dir)

    if output_dir_path.exists():
      dt_now = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
      dt_str = dt_now.isoformat()
      new_folder_name = str(output_dir_path) + "_" + dt_str
      output_dir_path = Path(new_folder_name)

    output_dir_path.parent.mkdir(parents=True, exist_ok=True)
    self.output_dir_path = output_dir_path

  def download_image(self, url, filename):
    output_path = self.img_dir_path + "/" + filename
    response = requests.get(url)
    if response.status_code == 200:
        with open(output_path, 'wb') as file:
            file.write(response.content)

  @staticmethod
  def show_mirador(output_path):
    print("\n認識結果は以下です。")
    path = f'/?manifest={output_path.replace("/content/docs", "")}&annotationState=1' # .replace("/content/docs", "")
    # output.serve_kernel_port_as_window(PORT, path=path) # , path=f'/?manifest={output_path.replace("/content", "")}&annotationState=1'
    output.serve_kernel_port_as_iframe(PORT, path=path, width="100%", height="600px")

  @staticmethod
  def parse_xml(xml_file):
    """指定されたXMLファイルをパースし、ルート要素を返します。"""
    tree = ET.parse(xml_file)
    return tree.getroot()

  @staticmethod
  def collect_pages_by_side(root, side):
      """
      XMLツリーのルートから、特定のサイド（'L'または'R'）に対応するページ要素を収集し、
      その要素の名前をキーとした辞書を返します。
      """
      return {page.get('IMAGENAME').replace(f'_{side}.jpg', ''): page
              for page in root.findall(".//PAGE")
              if f'_{side}.jpg' in page.get('IMAGENAME', '')}

  @staticmethod
  def remove_pages(root, pages):
      """
      指定されたページ要素をXMLツリーのルートから削除します。
      """
      for page in pages.values():
          root.remove(page)

  @staticmethod
  def merge(xml_file):
    """
    指定されたXMLファイル内のページをマージします。
    '_L.jpg'を名前に含むページに、対応する'_R.jpg'ページの内容を追加し、
    最後に'_R.jpg'ページを削除します。変更されたXMLは元のファイル名から".sorted"を削除したファイルに保存されます。
    """
    root = Task.parse_xml(xml_file)
    l_pages_dict = Task.collect_pages_by_side(root, 'L')
    r_pages_dict = Task.collect_pages_by_side(root, 'R')

    # _L.jpg を含む PAGE に _R.jpg の情報を追加
    for key, r_page in r_pages_dict.items():
        if key in l_pages_dict:
            l_page = l_pages_dict[key]

            l_page = l_pages_dict[key]
            l_width = int(l_page.get('WIDTH'))

            # _R.jpg の LINE 要素の X 属性を更新
            for line in r_page.findall(".//LINE"):
                line_x = int(line.get('X'))
                line.set('X', str(line_x + l_width))

            for elem in r_page.findall(".//POLYGON"):
                points = elem.attrib['POINTS']
                points_list = points.split(',')
                # x座標に新しいWIDTHを追加
                new_points_list = [str(int(points_list[i]) + l_width) if i % 2 == 0 else points_list[i] for i in range(len(points_list))]
                elem.attrib['POINTS'] = ','.join(new_points_list)

            # _R.jpg の子要素を _L.jpg の PAGE 要素に移動
            for elem in r_page:
                l_page.append(elem)

            # _L.jpg の IMAGENAME 属性を更新
            l_page.attrib['IMAGENAME'] = l_page.attrib['IMAGENAME'].replace('_L.jpg', '.jpg')

            #
            l_page.attrib['WIDTH'] = str(int(l_page.attrib['WIDTH']) + int(r_page.attrib['WIDTH']))

    Task.remove_pages(root, r_pages_dict)

    # XMLファイルを保存
    ET.ElementTree(root).write(xml_file.replace(".sorted", ""))

  def run_ocr(self):
    !python main.py infer "{str(self.temporary_directory)}" "{str(self.output_dir_path)}" -s s -p {str(self.process_range)}

  def save_extra(self, extra):
    if not extra:
      return

    output_dir = str(self.output_dir_path)
    task_id = str(self.task_id)
    output = f"{output_dir}/{task_id}/extra.json"

    with open(output, 'w') as f:
      json.dump(extra, f, indent=2)

  def copy_xml(self):
    # copy_xml
    output_dir = str(task.output_dir_path)
    task_id = str(task.task_id)

    xml_path = f"{output_dir}/{task_id}/xml/*.xml"
    xml_files = glob.glob(xml_path)

    for xml_file in xml_files:
      if ".sorted" in xml_file:
        !cp {xml_file} {xml_file.replace(".sorted", "")}

      """
      指定されたXMLファイル内のページをマージします。
      '_L.jpg'を名前に含むページに、対応する'_R.jpg'ページの内容を追加し、
      最後に'_R.jpg'ページを削除します。変更されたXMLは元のファイル名から".sorted"を削除したファイルに保存されます。
      """
      Task.merge(xml_file)

  def run_common_pipeline(self, services=None, extra = None):
    %cd {self.project_dir}

    ###

    task = self

    root_path =task.docs_dir
    output_dir = str(task.output_dir_path)

    task_id = str(task.task_id)

    ####

    self.run_ocr()

    # copy_xml
    self.copy_xml()

    IIIF.ndl_ocr_post_process_simple(root_path, output_dir, task_id, services=services)

    ###

    self.save_extra(extra)

    ###

    client = PdfClient()
    output = f"{output_dir}/{task_id}/pdf/{task_id}.pdf"
    input_path = f"{root_path}/iiif/{task_id}/manifest_a.json"
    client.convert_iiif2pdf(output, iiif_manifest_path=input_path, image_download_dir=f"{root_path}/files/{task_id}/img")

    output_text = f"{output_dir}/{task_id}/pdf/{task_id}_text.pdf"
    client.convert_iiif2pdf(output_text, iiif_manifest_path=input_path, image_download_dir=f"{root_path}/files/{task_id}/img", default_alpha=0.4)

    ###

    message(self.google_drive_path)

    ###

    output_manifest_alto_path = f"{root_path}/iiif/{task_id}/manifest_o.json"
    Task.show_mirador(output_manifest_alto_path)

    ### コピー

    iiif_in = f"{root_path}/iiif/{task_id}"
    iiif_out = f"{output_dir}/{task_id}/iiif"

    !cp -rp {iiif_in} {iiif_out}

    # 画像の削除
    !rm -rf {output_dir}/{task_id}/pred_img
    !rm -rf {output_dir}/{task_id}/img

    # google driveに移動
    self.move_files_to_google_drive()

  def move_files_to_google_drive(self):
    # google driveに移動
    output_dir = str(task.output_dir_path)
    task_id = str(task.task_id)
    content_path = f"{output_dir}/{task_id}"

    dirs = glob.glob(f"{content_path}/*")
    google_drive_path = self.google_drive_path
    for dir in dirs:

      !mv {dir} {google_drive_path}

  def process_image_from_url(self, url):

    self.run_common_pipeline()

  def prepare_pdf(self, pdf_path_, 画像の長辺をリサイズするサイズ = -1):
    # pdfのパス
    pdf_path = Path(pdf_path_)

    # 変換
    img_dir = self.img_dir_path
    os.makedirs(img_dir, exist_ok=True)

    # PDFを開く
    pdf_document = fitz.open(str(pdf_path))

    for page_num, page in enumerate(pdf_document, 1):
        # 高解像度でレンダリング（300 DPI相当）
        zoom = 300 / 72  # 72 DPIがデフォルト
        mat = fitz.Matrix(zoom, zoom)
        pix = page.get_pixmap(matrix=mat, alpha=False)

        # PILイメージに変換
        img_data = pix.tobytes("jpeg")
        img = Image.open(io.BytesIO(img_data))

        # リサイズ処理
        画像の長辺をリサイズするサイズ = int(画像の長辺をリサイズするサイズ)

        if 画像の長辺をリサイズするサイズ > 0:
            width, height = img.size
            if max(width, height) > 画像の長辺をリサイズするサイズ:
                ratio = 画像の長辺をリサイズするサイズ / max(width, height)
                new_width = int(width * ratio)
                new_height = int(height * ratio)

                try:
                    img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
                except AttributeError:
                    img = img.resize((new_width, new_height), Image.LANCZOS)

        # 保存
        target_name = Path(img_dir) / f"{page_num:04}.jpg"
        img.save(target_name, quality=95, optimize=True)

    pdf_document.close()

# google colab依存
from google.colab import files
def upload_get_input_file():
  uploaded = files.upload()
  input_file = next(iter(uploaded))
  return input_file

<br/><br/><br/><br/>

## 2.設定

以下、入力方式によって適切なものを選んでください。

- 画像
  - [単一の画像ファイルのURLを指定する場合](#scrollTo=8qA-c9Fto5Hz)
  - [単一の画像ファイルをアップロードする場合](#scrollTo=wVp2gJJLudLK)
  - [複数の既にダウンロード済みの画像ファイルを対象にする場合](#scrollTo=DyMdxEwt99TU)
- PDF
  - [単一のPDFファイルのURLを指定する場合](#scrollTo=xhaiIjgilpu5)
  - [単一のPDFファイルをアップロードする場合](#scrollTo=PAp-4c_2yw08)
  - [単一の既にダウンロード済みのPDFファイルを対象にする場合](#scrollTo=_rSjnrd2005J)
- IIIF
  - [IIIFマニフェストファイルのURLを指定する場合](#scrollTo=BQFx7V7iUDtO)

<br/><br/><br/><br/>

## 画像

### 単一の画像ファイルのURLを指定する場合

- url: 画像ファイルのURL
- 出力フォルダ: 出力するフォルダへのパス
<!-- - ruby: ルビのテキスト化を行うか否か -->

入力サンプル：「校異源氏物語」（国立国会図書館所蔵）

In [ ]:
#@title 実行

# url = "https://www.dl.ndl.go.jp/api/iiif/3437686/R0000006/full/1024,/0/default.jpg" #@param {type:"string"}
url = "https://dl.ndl.go.jp/api/iiif/3437686/R0000004/full/full/0/default.jpg" #@param {type:"string"}
出力フォルダ = "/content/drive/MyDrive/ndlocr_v2/output/image_url" #@param {type:"string"}
process = "\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)" #@param ["\u3059\u3079\u3066: \u30CE\u30C9\u5143\u5206\u5272,\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u6587\u5B57\u8A8D\u8B58(OCR)"]
ruby = False
# #@param {type:"boolean"}

task = Task(出力フォルダ, process, ruby)
task.download_image(url, f"{str(1).zfill(4)}.jpg")
task.run_common_pipeline()

In [ ]:
# @title すべてのセルを実行するときに停止させるための処理
assert False, "このセルで実行を停止しました。"

<br/><br/><br/>

### 単一の画像ファイルをアップロードする場合

- 出力フォルダ: 出力するフォルダへのパス
- ruby: ルビのテキスト化を行うか否か

以下の設定の再生ボタンを押すと、ファイルのアップロードフォームが表示されます。

In [ ]:
#@title 実行
出力フォルダ = "/content/drive/MyDrive/ndlocr_v2/output/image_local" #@param {type:"string"}
process = "レイアウト抽出,文字認識(OCR)" #@param ["\u3059\u3079\u3066: \u30CE\u30C9\u5143\u5206\u5272,\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u6587\u5B57\u8A8D\u8B58(OCR)"]
ruby = False
# #@param {type:"boolean"}

task = Task(出力フォルダ, process, ruby)

# google colab依存
from google.colab import files
def upload_get_input_file():
  uploaded = files.upload()
  input_file = next(iter(uploaded))
  return input_file

input_file = upload_get_input_file()

# 画像の移動
opath = task.img_dir_path + "/" + f"{str(1).zfill(4)}.jpg"
!mv "{input_file}" "{opath}"

task.run_common_pipeline()

<br/><br/><br/>

### 複数の既にダウンロード済みの画像ファイルを対象にする場合

- 入力フォルダ: 入力するフォルダのパス
  - 指定したフォルダの下にimgフォルダを用意し、その中に画像を格納してください。
- 出力フォルダ: 出力するフォルダへのパス

In [ ]:
#@title 実行

import glob

入力フォルダ = "/content/drive/MyDrive/ndlocr_v2/input" #@param {type:"string"}
出力フォルダ = "/content/drive/MyDrive/ndlocr_v2/output/image_local" #@param {type:"string"}
拡張子 = "*" #@param {type:"string"}
process = "レイアウト抽出,文字認識(OCR)" #@param ["\u3059\u3079\u3066: \u30CE\u30C9\u5143\u5206\u5272,\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u6587\u5B57\u8A8D\u8B58(OCR)"]
ruby = False
# #@param {type:"boolean"}

task = Task(出力フォルダ, process, ruby)

入力フォルダ = str(Path(入力フォルダ))

files = glob.glob(f"{入力フォルダ}/img/*.{拡張子}")
files.sort()

filename_change_map = {}

for i in tqdm(range(len(files))):
  # 画像のコピー
  input_file = files[i]
  filename = f"{str(i + 1).zfill(4)}.jpg"
  output_file = f"{task.temporary_directory}/img/{filename}"
  !cp "{input_file}" "{output_file}"
  filename_change_map[filename] = input_file.split("/")[-1]

task.run_common_pipeline(extra=filename_change_map)

<br/><br/><br/><br/>

## PDF

<br/><br/><br/>

### 単一のPDFファイルのURLを指定する場合

- url: PDFファイルのURL
- 出力フォルダ: 出力するフォルダへのパス
<!-- - ruby: ルビのテキスト化を行うか否か -->

入力サンプル：「東洋学芸雑誌」（人間文化研究機構国立国語研究所所蔵）

In [ ]:
#@title 設定
url = "https://dglb01.ninjal.ac.jp/ninjaldl/toyogakuge/001/PDF/tygz-001.pdf" #@param {type:"string"}
出力フォルダ = "/content/drive/MyDrive/ndlocr_v2/output/pdf_url" #@param {type:"string"}
process = "\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)" #@param ["\u3059\u3079\u3066: \u30CE\u30C9\u5143\u5206\u5272,\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u6587\u5B57\u8A8D\u8B58(OCR)"]
ruby = False
画像をリサイズする際の長辺の長さ = 1200  #@param {type:"integer"}

# #@param {type:"boolean"}

task = Task(出力フォルダ, process, ruby)

# pdfのパス
pdf_path = Path(f"/content/tmp/{task.task_id}.pdf")

# ダウンロード
os.makedirs(os.path.dirname(pdf_path), exist_ok=True)
!curl {url} -o {pdf_path} --insecure

task.prepare_pdf(pdf_path, 画像をリサイズする際の長辺の長さ)

task.run_common_pipeline()

<br/><br/><br/>

### 単一のPDFファイルをアップロードする場合

- 出力フォルダ: 出力するフォルダへのパス
<!-- - ruby: ルビのテキスト化を行うか否か -->

以下の設定の再生ボタンを押すと、ファイルのアップロードフォームが表示されます。

In [ ]:
#@title 設定

出力フォルダ = "/content/drive/MyDrive/ndlocr_v2/output/pdf_local" #@param {type:"string"}
process = "\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)" #@param ["\u3059\u3079\u3066: \u30CE\u30C9\u5143\u5206\u5272,\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u6587\u5B57\u8A8D\u8B58(OCR)"]
ruby = False
# #@param {type:"boolean"}
画像をリサイズする際の長辺の長さ = 1200  #@param {type:"integer"}

task = Task(出力フォルダ, process, ruby)

pdf_path = upload_get_input_file()

task.prepare_pdf(pdf_path, 画像をリサイズする際の長辺の長さ)
task.run_common_pipeline()

<br/><br/><br/>

### 単一の既にダウンロード済みのPDFファイルを対象にする場合

- input_file: PDFファイルのパス
- 出力フォルダ: 出力するフォルダへのパス
<!-- - ruby: ルビのテキスト化を行うか否か -->

In [ ]:
#@title　設定

# input_file = "/content/drive/MyDrive/ndl_ocr2/input/pdfs/tygz-001.pdf" #@param {type:"string"}
入力ファイル = "/content/drive/MyDrive/ndlocr_v2/input/pdfs/2M5-4.pdf" #@param {type:"string"}
出力フォルダ = "/content/drive/MyDrive/ndlocr_v2/output/pdf_local" #@param {type:"string"}
process = "\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)" #@param ["\u3059\u3079\u3066: \u30CE\u30C9\u5143\u5206\u5272,\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u6587\u5B57\u8A8D\u8B58(OCR)"]
ruby = False
# #@param {type:"boolean"}

task = Task(出力フォルダ, process, ruby)
task.prepare_pdf(入力ファイル)
task.run_common_pipeline()

<br/><br/><br/><br/>

## IIIF

<br/>

### IIIFマニフェストファイルのURLを指定する場合

- IIIFマニフェストファイルのURL: IIIFマニフェストファイルのURL
- 出力フォルダ: 出力するフォルダへのパス
- 開始コマ数: 処理を開始するコマ。デフォルトは1。
- 終了コマ数: 処理を終了するコマ。デフォルトは5。-1にするとすべて。
- 画像ダウンロードの間隔_秒数: 画像ダウンロードの間隔（秒数）

入力サンプル：「校異源氏物語」（国立国会図書館所蔵）

In [ ]:
#@title 設定
import time
from tqdm import tqdm

IIIFマニフェストファイルのURL = "https://dl.ndl.go.jp/api/iiif/3437686/manifest.json"#@param {type:"string"}

出力フォルダ = "/content/drive/MyDrive/ndlocr_v2/output/iiif" #@param {type:"string"}

開始コマ数 =   1 #@param {type:"number"}
終了コマ数 =   11 #@param {type:"number"}
画像ダウンロードの間隔_秒数 =   1 #@param {type:"number"}

process = "\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)" #@param ["\u3059\u3079\u3066: \u30CE\u30C9\u5143\u5206\u5272,\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u6587\u5B57\u8A8D\u8B58(OCR)"]
ruby = False
# #@param {type:"boolean"}

task = Task(出力フォルダ, process, ruby)

df = requests.get(IIIFマニフェストファイルのURL).json()
context = df["@context"]

targets = []
services = []

if context == "http://iiif.io/api/presentation/2/context.json":
  canvases = df["sequences"][0]["canvases"]

  for i in range(len(canvases)):
    canvas = canvases[i]

    if 終了コマ数 == -1 or not (開始コマ数 - 1 <= i  and i < 終了コマ数):
      continue

    url = canvas["images"][0]["resource"]["@id"]

    targets.append({
        "url": url,
        "filename": f"{str(i + 1).zfill(4)}.jpg"
    })

    services.append(canvas["images"][0]["resource"]["service"])

else:
  pass

# print(services)

for target in tqdm(targets):
  task.download_image(target["url"], target["filename"])
  time.sleep(画像ダウンロードの間隔_秒数)

task.run_common_pipeline(services=services)